## Introduction

In this class, we will learn:
1. How to import the Pandas library.
2. How to load data from different files (CSV, Excel, JSON).
3. How to extract specific rows, columns, and values.
4. How to modify the data and save it again.
5. How to use NumPy operations and a simple for-loop.
6. How to pick X and Y and plot.
7. Basic data **cleaning**: handle missing values and drop low-variation columns.

**Note for Google Colab users:** If you run this in Colab, you can either upload files with `files.upload()` or mount Google Drive with `drive.mount('/content/drive')`. See the Colab section below.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Main folder path for your data. Adjust as needed.
data_path = "../dummy_data/"

## Notes for Google Colab

**Option A: Upload files each time**

```python
from google.colab import files
uploaded = files.upload()  # Choose data.csv, data.xlsx, data.json
```

After upload, files live in the current folder `/content/`, so set:

```python
data_path = ""  # empty, files are next to the notebook
```

**Option B: Mount Google Drive (persistent storage)**

```python
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/dummy_data/"  # put your folder in Drive
```

### Path tips (`../`)
- `"file.csv"` → file in the **current folder**
- `"../file.csv"` → file in the **parent folder** (go back once)
- `"../../file.csv"` → go back **two folders up**
- `"../dummy_data/data.csv"` → go back one folder, then into `dummy_data/`

## Loading Data (CSV / Excel / JSON)

We will start with CSV. We use the `data_path` variable so paths are easy to switch (local, Colab, or Drive).

In [ ]:
# CSV example
df = pd.read_csv(data_path + "data.csv")
print(df.head())

In [ ]:
# Excel example (uncomment if you have data.xlsx)
# df_excel = pd.read_excel(data_path + "data.xlsx")
# print(df_excel.head())

# JSON example (uncomment if you have data.json)
# df_json = pd.read_json(data_path + "data.json")
# print(df_json.head())

### Are CSV and Excel different to read?
- **CSV**: plain text, use `pd.read_csv(...)`. Options you often use: `sep`, `header`, `names`, `dtype`, `na_values`.
- **Excel**: spreadsheet format, use `pd.read_excel(...)`. Extra options: `sheet_name` (which sheet), `engine` (sometimes needed), date parsing per sheet, etc.

You **do not** need to “construct” a DataFrame manually—both functions return a DataFrame directly. You only adjust parameters if the file has special formatting.

## Basic Exploration

In [ ]:
print(df.head())
print(df.shape)
print(df.columns)
print(df.info())

## Extracting Columns

In [ ]:
# One column as DataFrame
one_col_df = df[["Name"]]
print(one_col_df.head())

# Drop one column (copy into a new df)
no_city_df = df.drop(columns=["City"])
print(no_city_df.head())

## Filtering

In [ ]:
filtered = df[df["Age"] > 20]
print(filtered)

## Modifying Data (add a new column)

In [ ]:
df["Age_plus_10"] = df["Age"] + 10
print(df.head())

## NumPy Operations (using the data)

In [ ]:
ages = df["Age"].to_numpy()

# NumPy stats
age_mean = np.mean(ages)
age_std_pop = np.std(ages)
age_std_sample = np.std(ages, ddof=1)
print("Mean Age:", age_mean)
print("Std Dev Age (population):", age_std_pop)
print("Std Dev Age (sample):", age_std_sample)

# Simple for-loop example: manual average
total = 0
count = 0
for a in ages:
    total += a
    count += 1
manual_avg = total / count if count > 0 else float("nan")
print("Manual average (for-loop):", manual_avg)

## Selecting X and Y (two ways)

In [ ]:
# Way 1: tuple assignment in one line
x, y = df["Age"].to_numpy(), df["Age_plus_10"].to_numpy()
print(x[:5], y[:5])

# Way 2: separate assignments
x = df["Age"].to_numpy()
y = df["Age_plus_10"].to_numpy()
print(x[:5], y[:5])

## Plot X vs Y

In [ ]:
plt.figure()
plt.plot(x, y, marker="o")
plt.xlabel("Age (X)")
plt.ylabel("Age_plus_10 (Y)")
plt.title("Age vs Age_plus_10")
plt.grid(True)
plt.show()

## Cleaning Data (Handling Missing Values)

Sometimes the dataset has empty cells. Pandas shows them as NaN (Not a Number).
We can:
- Drop rows with missing values.
- Fill missing values with something (like a word or a number).

In [ ]:
# Work on a copy
df_clean = df.copy()

# Add a missing value for demonstration
df_clean.loc[2, "City"] = None
print(df_clean)

# Drop rows with any missing values (new DataFrame)
dropped = df_clean.dropna()
print("After dropping rows with NaN:\n", dropped)

# Fill missing values with a default (new DataFrame)
filled = df_clean.fillna({"City": "Unknown"})
print("After filling NaN:\n", filled)

## Cleaning Data (Drop Low-Variation Columns)

If a column has very few unique values, it may not help. Example: a 'Size' column where almost all values are 50.

In [ ]:
df_clean2 = df.copy()
df_clean2["Size"] = [50, 50, 50, 60, 50]
print(df_clean2)
print("Unique values in Size:", df_clean2["Size"].unique())

if df_clean2["Size"].nunique() <= 2:
    df_clean2 = df_clean2.drop(columns=["Size"])
    print("Dropped 'Size' column because it had too few unique values")

print(df_clean2)

## Inplace vs Not Inplace

- Many Pandas methods have `inplace=`.
- `inplace=True` **modifies the DataFrame directly** and returns `None`.
- `inplace=False` (default) **returns a new DataFrame** and leaves the original unchanged.

### Examples
```python
# Not inplace (preferred for beginners):
no_city_df = df.drop(columns=["City"])  # df stays the same

# Inplace (df is changed):
df_copy = df.copy()
df_copy.drop(columns=["City"], inplace=True)
```

### What does `index=False` / `index=True` mean when saving?
- DataFrames have an index (row labels). When saving to CSV/Excel:
  - `index=False` → **do not** write the index column to the file (common for clean files).
  - `index=True` → **include** the index column in the output file.

## Build a New DataFrame from Results

In [ ]:
xy_df = pd.DataFrame({"Age_X": df["Age"].to_numpy(), "AgePlus10_Y": df["Age_plus_10"].to_numpy()})
subset_df = pd.concat([df[["Name", "City"]], xy_df], axis=1)
print(subset_df.head())

summary_df = pd.DataFrame({
    "Mean_Age": [np.mean(df["Age"])],
    "Std_Age_pop": [np.std(df["Age"])],
    "Std_Age_sample": [np.std(df["Age"], ddof=1)]
})
print(summary_df)

## `axis` in Pandas

* **`axis=0` → rows** (goes *down*).
* **`axis=1` → columns** (goes *across*).

Think:

* `0` = vertical (rows).
* `1` = horizontal (columns).

---

## Example: Concatenating along columns (`axis=1`)

In [ ]:
import pandas as pd

df1 = pd.DataFrame({"A": [1, 2, 3]})
df2 = pd.DataFrame({"B": [10, 20, 30]})

result = pd.concat([df1, df2], axis=1)
print(result)



Here they align **by rows** (same index), so we get new columns.



## Example: Concatenating along rows (`axis=0`)

In [ ]:

df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"A": [3, 4]})

result = pd.concat([df1, df2], axis=0)
print(result)




Here they stack **on top of each other**, row-wise. Notice the index repeats—you may want `ignore_index=True`.

In [ ]:

result = pd.concat([df1, df2], axis=0, ignore_index=True)
print(result)


---

## What if columns don’t match?

Yes — if one DataFrame has more columns than the other:

In [ ]:
df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"B": [10, 20]})

result = pd.concat([df1, df2], axis=0)
print(result)



Values that don’t exist become **`NaN` (None/empty)**.

## Saving Results

Works the same locally, in Colab `/content/`, or in Drive `/content/drive/MyDrive/...` depending on `data_path`. Set `index=False` to avoid writing the index.

In [ ]:
# Save modified main df
df.to_csv(data_path + "data_modified.csv", index=False)
df.to_excel(data_path + "data_modified.xlsx", index=False)
df.to_json(data_path + "data_modified.json", orient="records")

# Save analysis outputs
subset_df.to_csv(data_path + "final_xy_subset.csv", index=False)
summary_df.to_csv(data_path + "summary_stats.csv", index=False)

## Summary
- Use `data_path` to control where files are read/written. 
    - Its better as a form of control

- CSV vs Excel: different readers, same result (a DataFrame). Excel has sheets.
- Use `[["col"]]` to keep a DataFrame, `.drop(columns=[...])` to remove.

- Filter with boolean expressions like `df[df["Age"] > 20]`.
- Add/modify columns directly, e.g., `df["Age_plus_10"] = df["Age"] + 10`.

- NumPy: quickly compute stats like mean/std; simple loops are fine for teaching.

- Clean data: handle NaN by dropping or filling; drop low-variation columns.

- `inplace=True` changes the DataFrame; otherwise you get a new one.
- `index=False` when saving = don't write the index column.

- **`axis=1` → add columns** side by side.
- **`axis=0` → add rows** stacked one under the other.
    - **Mismatched columns** → missing values filled with `NaN`.
    - Use `ignore_index=True` if you want clean row numbering after `axis=0`.